## Importing Libraries

In [1]:
import os
import io

"I have imported the word embeddings text rank code as textrank_new"
import textrank_new

C:\Users\hp\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.
C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### TestDoc :
Every test file is an object with these attributes 

In [ ]:
class TestDoc:
    def __init__(self, name):
        self.name = name
        self.text = ''
        self.keywords = []

In [2]:
# reading a directory with test documents
os.chdir("...")
input_dir = os.getcwd()

## Funtion evaluate :
Takes in the input directiry . Reads every test file and then its corresponding .key file . Calls another function to get the keywords generated by another algorithms and then compares them with the manual keywords.

In [3]:
def evaluate( input_dir , object_algo , top = 5):
    
    test_set = {} 
    
    for docs in os.listdir(input_dir):
        file_reader = io.open(os.path.join(input_dir,docs), 'r' , encoding="iso-8859-1")
        file_name = docs[:-4]
        if file_name not in test_set:
            d = TestDoc(file_name)
        else:
            d = test_set[file_name]

        if not docs.endswith(".txt"):
            continue
        
        text = file_reader.read()
        d.text = text
   
        # get document keywords
        if file_name == '~$C-1':
            continue
            
        file_reader = open(os.path.join(input_dir,file_name + ".key"), 'r')
        manual_keywords = file_reader.read()
        for line in manual_keywords.split('\n'):
            line = line.rstrip().lower()
            if len(line) > 0:
                if '\t' in line:
                    d.keywords.append(line[0:line.find('\t')])
                else:
                    d.keywords.append(line)

        # add document to test set
        test_set[file_name] = d
   

    total_precision = 0
    total_recall = 0

    
    for test_doc in test_set.values():
        #if test_doc.name == '~$C-1':
           # continue
        print('document', test_doc.text)
        print(len(test_doc.keywords), 'manual keywords: ', test_doc.keywords)
        ## A SEPERATE FUNCTION IS CALLED TO RUN THE ALGORITHM AND GET THE KEYWORDS GENERATED 
        
        keywords = get_keywords(input_dir , test_doc.name)
        print('Algorithm keywords:', keywords)
        num_manual_keywords = len(test_doc.keywords)
        correct = 0
        for i in range(0,min(top, len(keywords))):
            if keywords[i][0] in set(test_doc.keywords):
                correct += 1
        total_precision += correct/float(len(keywords))
        total_recall += correct/float(num_manual_keywords)
        print('correct:', correct, 'out of', num_manual_keywords)

    
    avg_precision = round(total_precision*100/float(len(test_set)), 2)
    avg_recall = round(total_recall*100/float(len(test_set)), 2)

    avg_fmeasure = round(2*avg_precision*avg_recall/(avg_precision + avg_recall), 2)

    print("Precision", avg_precision, "Recall", avg_recall, "F-Measure", avg_fmeasure)


### Helper function

In [4]:
def get_keywords(input_dir ,file_name , algo = "textrank"):
    
    import spacy
    nlp = spacy.load('en_core_web_sm')
    text = io.open(os.path.join(input_dir,file_name + ".txt"), 'r' , encoding="iso-8859-1").read() # open a document
    doc = nlp(text) 

    " Further algorithms and their funciton calls can be added and passed as in argument to this function"
    if algo == "textrank":
        terms = textrank_new.key_terms_from_semantic_network( doc, normalize='lemma',
                                    window_width=2, edge_weighting='embedding',
                                    ranking_algo='pagerank', join_key_words = True,
                                    n_keyterms=10)  
    
    
    
    return terms 

In [ ]:
evaluate( input_dir , textrank_new , top = 5)